<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Python/chap12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第12章 パターンマイニング

In [ ]:
!pip list | grep mlxtend

In [ ]:
# mlxtendを最新版 0.23.0 (2023/10/23現在) に更新
!pip install -U mlxtend

## 頻出項目集合抽出

Aprioriアルゴリズムの実装

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

### 例題12.1

例題12.1 のデータを読み込んで、pandasのDataFrameに変換します。項目がアルファベット順にソートされていることに注意。

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [ ]:
dataset = [
    ['Milk', 'Bread', 'Butter'],
    ['Milk', 'Bread', 'Jam'],
    ['Milk', 'Magazine'],
    ['Bread', 'Butter'],
    ['Milk', 'Bread', 'Butter', 'Jam'],
    ['Magazine'],
    ['Milk', 'Bread', 'Jam', 'Magazine'],
    ['Jam']]

In [ ]:
# 警告が出る場合は以下で抑制
import warnings
warnings.simplefilter('ignore')

In [ ]:
# 疎行列形式の表現を、真偽値を値とする行列に変換
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Aprioriアルゴリズムで3事例以上出現する項目集合を抽出

In [ ]:
freq = apriori(df, min_support= 3/len(df), use_colnames=True)
freq

## 規則抽出


### 例題12.2

confidenceが0.7以上の規則を抽出します。

In [ ]:
ar = association_rules(freq, metric='confidence', min_threshold=0.7)

In [ ]:
ar[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

抽出された規則をplotlyで可視化するために、データ形式を変換します。この事例ではスコアが同じ規則が複数あって、プロットが重なってしまっていることに注意してください。

In [ ]:
alist = []
clist = []
for a, c in zip(ar['antecedents'], ar['consequents']):
  alist.append(','.join(a))
  clist.append(','.join(c))
ar2 = ar.drop(['antecedents','consequents'], axis=1)
ar2['antecedents'] = alist
ar2['consequents'] = clist
ar2

In [ ]:
import plotly.express as px
fig = px.scatter(ar2, x = 'support', y = 'confidence', color='lift', hover_data=['antecedents','consequents'], range_x=[0.3, 0.6])
fig.show()

### 例題12.3

supermarket.arffでFPGrowth（Aprioriの高速化版）を試してみます。データをダウンロードし、scipyのloadarffでWekaのarff形式のデータを読み込みます。そしてpandasのDataFrameで値を文字t/?から真偽値True/Falseとし、最終列のtotalは除きます。

In [ ]:
!wget https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff

In [ ]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket.arff')
df = pd.DataFrame(data)
df2 = df.replace({b'?':False, b't':True})
df2 = df2.drop('total', axis=1)
df2

FPGrowthで頻出項目集合を求め、association_rulesで規則を抽出します。

In [ ]:
freq2 = fpgrowth(df2, min_support=0.3, use_colnames=True)

In [ ]:
ar = association_rules(freq2, metric='lift', min_threshold=1.2)

plotlyで可視化するためのデータ変換

In [ ]:
alist = []
clist = []
for a, c in zip(ar['antecedents'], ar['consequents']):
  alist.append(','.join(a))
  clist.append(','.join(c))
ar2 = ar.drop(['antecedents','consequents'], axis=1)
ar2['antecedents'] = alist
ar2['consequents'] = clist
ar2

各規則がsupportとconfidenceの2次元空間にプロットされ、lift値は色の明るさで示されます。点にカーソルを合わせると、その点が表現している規則とそのスコアが表示されます。

In [ ]:
import plotly.express as px
fig = px.scatter(ar2, x = 'support', y = 'confidence', color='lift', hover_data=['antecedents','consequents'])
fig.show()

## 行列分解

小規模な映画評価データを使って、行列分解を行います。



ライブラリの読み込み

In [ ]:
import numpy as np
from sklearn.decomposition import NMF

<a href="http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/">参考ページ</a>のデータ例を使用します。行がユーザ(5人)、列が映画(4作品)、数値が1-5の5段階評価で、0は評価なしです。

In [ ]:
X = np.array([
    [5,3,0,1],
    [4,0,0,1],
    [1,1,0,5],
    [1,0,0,4],
    [0,1,5,4]
])

ここでは、非負値行列分解[NMF](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)を使います。NMFはXを非負の行列WとHに分解します。引数n_componentsは潜在変数の次元数です。

In [ ]:
model = NMF(n_components = 2)
W = model.fit_transform(X)
H = model.components_

空所の値を予測してみます。似たようなユーザを参考に、埋められた値の妥当性を評価してみてください。

In [ ]:
np.set_printoptions(suppress=True)
np.dot(W,H)

人を表す2次元ベクトルを表示します。同じような評価をする人が似たベクトルになっていることを確認してください。

In [ ]:
W

映画を表す2次元ベクトルを表示します。同じような評価をされる映画がないので、それぞれ異なったベクトルになっていることを確認してください。

In [ ]:
H.T

## 練習問題

1. a prioriアルゴリズムによる規則抽出のコード例で求めた規則よりも、Lift値の高い規則を探すにはどうすればよいでしょうか。
1. 教科書p.226の演習問題12-3の手順に従い、MovieLensデータセットに対してscikit-surpriseを用いてNMFを行ってください。余力があれば、ハイパーパラメータを変更して、性能に与える影響を観察してください。

### 解答例

#### 練習問題1

FPGrowthでmin_supportを下げて多くの頻出項目集合を求め、association_rulesで規則を評価します。ただしmin_supportをあまり下げすぎると、あまり意味のない項目集合が出てきます。

In [ ]:
freq2 = fpgrowth(df2, min_support=0.2, use_colnames=True)
association_rules(freq2, metric="lift", min_threshold=1.5)

#### 練習問題2

In [ ]:
!pip install scikit-surprise

In [ ]:
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

# movielens-100k データの読み込み
X = Dataset.load_builtin('ml-100k', prompt=False)

In [ ]:
# 5-fold CVで、平均平方二乗誤差と平均絶対誤差を表示します。
cross_validate(NMF(), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

scikit-surprise NMFのデフォルトの圧縮次元数(n_factors)は15です。これを下げると、実行時間は早くなっていますが、性能は悪くなっています。

In [ ]:
cross_validate(NMF(n_factors=5), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)